In [1]:
import json
from pyld import jsonld
import os, re, sys
from glob import glob
from pprint import pprint

def rd(f):
    return json.load(open(f))

def wt(d,f):
    return json.dump(d,open(f,'w'),indent=4)

In [2]:
loc = '/Users/daniel.ellis/WIPwork/WCRP-universe/src-data/'
dirs = os.listdir(loc)
print(dirs)

['del.ipynb', '.DS_Store', 'source-type', 'consortium', 'license', 'archive', 'frequency', '_context_', 'activity', 'product', 'mip', 'grid-label', 'institution', 'resolution', '.github', 'realm']


In [4]:
section = f'{loc}realm'
js = glob(f'{section}/*.json')

pprint(rd(js[0]))


{'@context': '_context_',
 'description': 'Ocean Biogeochemistry',
 'id': 'ocnbgchem',
 'label': 'ocnBgchem',
 'long_label': 'Ocean Biogeochemistry',
 'type': ['wcrp:realm', 'universal']}


In [143]:
write = False



ctx = rd(glob(f'{section}/_context_')[0])['@context']

pprint(ctx)

ids = []
for i in ctx:
    if isinstance(i,dict):
        for j,k in i.items():
            
            assert '-' not in j
            
            if '@id' in str(k):
                ids.append(j)
            
print('links', ids)





# replace
for f in js:
    print(f)
    data = rd(f)

    data['id'] = data['id'].lower().replace('_','-')    
      
    # data['long_label'] = data['long_name']
    # del data['long_name']
    # data['label'] = data['id']
    
    # mt = {}
    # for i in data['mip_tables']:
    #     k = i['@id']
    #     del i['@id']
    #     mt[k] = dict(zip([d.lower().replace('-','_') for d in i.keys()],i.values()))
        
    # data['mip_tables'] = mt
    
    
    
    # fix keys to have no -
    data= dict(zip([d.lower().replace('-','_') for d in data.keys()],data.values()))
            
            
    stop = False
    

    for nm in ['name','cmip_acronym','kind','table_id']:
        if nm in data:
            if 'label' in data and data['label'] == data[nm]:
                del data['label']
            else: 
                assert 'label' not in data
            
            data['label'] = data[nm]
            del data[nm] 
                
                    
    # id fix
    for z in ids:
        
        # if z not in data: continue
        
        
        if isinstance(data[z],list):
            data[z] = [i.lower().replace('_','-') for i in data[z]]
            
        if isinstance(data[z],dict):
            values = []
            for p in data[z].values():
                if isinstance(p,dict):
                    # normal keys are a '_' 
                    p = dict(zip([d.lower().replace('-','_') for d in p.keys()],p.values()))
                values.append(p)
                
            # ids are a hypen
            data[z]= dict(zip([d.lower().replace('_','-') for d in data[z].keys()],values))
            
            


    # if 'source_id' in data:
    #     del data['source_id']
    
    

            
    if write:        
        if '_' in f.split('/')[-1]:
            rename=True
            os.remove(f)
            f = f"{section}/{data['id']}.json"
            print(f)
            
        wt(data,f)
        
        assert os.path.exists(f)
    
    pprint(data)

['../_context_',
 {'@base': 'https://wcrp-cmip.github.io/MIP-variables/variables/',
  '@vocab': 'https://wcrp-cmip.github.io/MIP-variables/variables/',
  'mip_tables': {'@context': 'https://wcrp-cmip.github.io/MIP-variables/tables/_context_',
                 '@reverse': 'https://wcrp-cmip.github.io/MIP-variables/tables/variable_entry'},
  'standard_name': {'@context': 'https://wcrp-cmip.github.io/CF/standard_name/_context_',
                    '@type': '@id'}}]
links ['standard_name']
/Users/daniel.ellis/WIPwork/MIP-variables/src-data/variables/gigyr.hfgeoubed.json
{'@context': '_context_',
 'cell_measures': 'area: areacellg',
 'cell_methods': 'area: time: mean where grounded_ice_sheet (comment: '
                 'mask=sfgrlf)',
 'comment': 'Upward geothermal heat flux per unit area into the base of '
            'grounded land ice. This is related to the geothermal heat flux '
            'out of the bedrock, but may be modified by horizontal transport '
            'due to run-off